# 메리루트(MarryRoute) chat_demo

**목표:** 파서 → 플래너 업데이트(메모리) → 카탈로그 추천/목록 → 미니 챗 데모 흐름을 한 파일에서 실행

## 0) 경로/모듈 확인

In [ ]:
# 필수 경로
DB_PATH = "marryroute.db"  # 프로젝트 루트에서 실행 권장

# 표시는 예쁘게
import json, sqlite3, textwrap, os, sys
print("Python:", sys.version)
print("Working dir:", os.getcwd())
print("DB exists:", os.path.exists(DB_PATH))

## 1) DB 스키마/테이블 간단 점검

In [ ]:
# 주요 테이블 목록 및 간단 점검
con = sqlite3.connect(DB_PATH)
con.row_factory = sqlite3.Row
tables = [r["name"] for r in con.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()]
print("Tables:", tables)

# vendor/offering 칼럼 확인
def pragma(table):
    try:
        rows = con.execute(f"PRAGMA table_info({table})").fetchall()
        return [{k: row[k] for k in row.keys()} for row in rows]
    except Exception as e:
        return str(e)

print("\nPRAGMA vendor:", json.dumps(pragma("vendor"), ensure_ascii=False, indent=2))
print("\nPRAGMA offering:", json.dumps(pragma("offering"), ensure_ascii=False, indent=2))

# 샘플 카운트
def count(table):
    try:
        return con.execute(f"SELECT COUNT(*) AS c FROM {table}").fetchone()["c"]
    except Exception as e:
        return str(e)

for t in ["user_profile","budget_pref","event","conversation_summary","vendor","offering"]:
    print(f"{t:>22} :", count(t))

## 2) 파서(parser.py) 스모크 테스트
- 다양한 문장을 파싱해서 날짜/지역/카테고리 예산/이벤트를 확인

In [ ]:
from parser import parse_text

tests = [
    "메이크업은 청담역이 좋아",
    "스튜디오는 청담역에서 찾을래",
    "드레스 300~400, 메이크업 120 이하, 스튜디오는 홍대입구역",
    "예식은 교대역에서 할거야",
    "본식은 10/26 12:00 교대",
]

for t in tests:
    print("\n--", t)
    print(json.dumps(parse_text(t), ensure_ascii=False, indent=2))

## 3) 플래너 업데이트(planner_update.py) — DB 반영 & 요약

In [ ]:
from planner_update import update_from_text, _fetch_state, build_summary_text

def show_state():
    st = _fetch_state(DB_PATH)
    print("요약:", build_summary_text(st))
    print(json.dumps(st, ensure_ascii=False, indent=2))

# 지역/예산/이벤트 반영
_ = update_from_text(DB_PATH, "스튜디오는 홍대입구역, 드레스 300~400, 메이크업 120 이하", dry_run=False)
_ = update_from_text(DB_PATH, "예식은 10/26 12:00 교대역", dry_run=False)
show_state()

# 카테고리 지역 '교체' 정책 확인
_ = update_from_text(DB_PATH, "스튜디오는 청담역으로", dry_run=False)
show_state()

# 날짜만 바꿔도 기존 위치/예산 유지 확인
_ = update_from_text(DB_PATH, "본식 날짜는 10/27로 바꿔줘", dry_run=False)
show_state()

## 4) 이벤트 레코드 확인 (SQL)

In [ ]:
rows = con.execute("SELECT event_id,type,title,date,time,location,budget_manwon FROM event ORDER BY event_id").fetchall()
for r in rows:
    print(dict(r))

## 5) 카탈로그 추천/목록 (catalog_bridge.py) — 지역 우선 → 가격 오름차순

In [ ]:
from catalog_bridge import recommend, catalog

# 선호를 설정해 둔 뒤 호출하세요. (위 3) 셀에서 선호가 반영됩니다.)
print("\n[추천: studio]")
print(json.dumps(recommend(DB_PATH, "studio", limit=5), ensure_ascii=False, indent=2))

print("\n[목록: hall]")
print(json.dumps(catalog(DB_PATH, "hall", limit=10), ensure_ascii=False, indent=2))

print("\n[추천: dress]")
print(json.dumps(recommend(DB_PATH, "dress", limit=5), ensure_ascii=False, indent=2))

print("\n[목록: makeup]")
print(json.dumps(catalog(DB_PATH, "makeup", limit=10), ensure_ascii=False, indent=2))

## 6) 미니 챗 데모(chat.py) 사용 가이드
터미널에서 실행하세요:
```bash
python chat.py
# 예시
You> 스튜디오는 청담역, 드레스 300~400, 메이크업 120 이하, 예식은 교대역
You> /추천 스튜디오
You> /목록 홀
You> /summary
You> /quit
```
- `/추천`, `/목록`은 `/recommend`, `/catalog`와 동일합니다.
- 정렬: **지역 우선 → 가격 오름차순(NULL은 뒤)**

## 7) 타임라인(timeline.py) 안내
- `timeline.py`가 있다면, 예식일 기준으로 기본 체크리스트를 생성/갱신하도록 호출합니다.
- 예시:
```bash
python timeline.py build --db marryroute.db
python timeline.py list  --db marryroute.db
```
- 기본 이벤트 기준 자동 생성 규칙(DEFAULT_PLAN)은 코드 내 상단 주석을 참고
